In [1]:
from my_trader import *

C:\ProgramData\Anaconda2\lib\site-packages\fix_yahoo_finance\__init__.py:43: DeprecationWarning: 
    Auto-overriding of pandas_datareader's get_data_yahoo() is deprecated and no longer available.
    Use pdr_override() to explicitly override it.
  DeprecationWarning)


In [22]:
robinhood = get_robinhood()
stock1 = "UVXY"
stock2 = "SPY"
window = 7
initial = 1200


price1 = robinhood.get_historical(stock1)
price2 = robinhood.get_historical(stock2)
price1 = price1.set_index("begins_at")
price2 = price2.set_index("begins_at")
price1 = price1.rename(columns={"close_price":stock1+"_close"})
price2 = price2.rename(columns={"close_price":stock2+"_close"})



price_table = pd.concat([price1[stock1+"_close"],price2[stock2+"_close"]],axis = 1)
price_table[stock1+"_close"] = price_table[stock1+"_close"].astype(float)
price_table[stock2+"_close"] = price_table[stock2+"_close"].astype(float)
price_table[stock1+"_log_ret"] = log(price_table[stock1+"_close"] / price_table[stock1+"_close"].shift(1))
price_table[stock2+"_log_ret"] = log(price_table[stock2+"_close"] / price_table[stock2+"_close"].shift(1))
price_table[stock1+"_log_ret_mv"] = price_table[stock1+"_log_ret"].rolling(window).mean()
price_table[stock2+"_log_ret_mv"] =price_table[stock2+"_log_ret"].rolling(window).mean()
price_table["relative"]=price_table[stock1+"_log_ret"]/price_table[stock2+"_log_ret"]
price_table["relative_mv"] = price_table["relative"].rolling(window).mean()
#price_table["relative_mv"]=price_table[stock1+"_log_ret_mv"]/price_table[stock2+"_log_ret_mv"]
price_table["z_score"] =( price_table["relative"]-price_table["relative_mv"])/price_table.relative.std()
price_table["trade"]=np.NaN
price_table["trade_signal"]=np.NaN
price_table["slope"]=np.NaN
for i in range(window-1,len(price_table)):
    price_table["slope"].iloc[i] = stats.linregress(price_table[stock1+"_log_ret"]\
                                                    .iloc[i-(window-1):i],price_table[stock2+"_log_ret"].iloc[i-(window-1):i])[0]
price_table["buy_line"] = np.NaN
for i in range(window-1,len(price_table)):
    if price_table.slope.iloc[i]>0.5:
        price_table["buy_line"].iloc[i] = -1.25
    elif price_table.slope.iloc[i]>0.75:
        price_table["buy_line"].iloc[i] -1.75
    else:
        price_table["buy_line"].iloc[i] = -2
price_table["sell_line"] = np.NaN
for i in range(window-1,len(price_table)):
    if price_table.slope.iloc[i]>0.5:
        price_table["sell_line"].iloc[i] = 1.25
    elif price_table.slope.iloc[i]>0.75:
        price_table["sell_line"].iloc[i] = 1.75
    else:
        price_table["sell_line"].iloc[i] = 2  
#set live trade signal and backtest 
for i in range(window-1,len(price_table)):
    if price_table.z_score.iloc[i] < price_table.buy_line.iloc[i]:
        price_table.trade.iloc[i] = 1
        price_table.trade_signal.iloc[i] = 1
    elif price_table.z_score.iloc[i] > price_table.sell_line.iloc[i]:
        price_table.trade.iloc[i] = 0
        price_table.trade_signal.iloc[i] = 0
    else:
        price_table.trade.iloc[i] = price_table.trade.iloc[i-1]

        
#price_table["cost_per_trade"] =abs( price_table[stock1+"_close"]+price_table[stock2+"_close"]*price_table["relative_mv"])
price_table[stock1+"_value"] = initial/(1+price_table["relative_mv"])
price_table[stock2+"_value"] = initial/(1+price_table["relative_mv"])*price_table["relative_mv"]
price_table[stock1+"_shares"] = price_table[stock1+"_value"]/price_table[stock1+"_close"]
price_table[stock2+"_shares"] = price_table[stock2+"_value"]/price_table[stock2+"_close"]

price_table["p_L"] = price_table[stock1+"_value"] * price_table[stock1+"_log_ret"]*price_table.trade +  \
price_table[stock2+"_value"] * price_table[stock2+"_log_ret"]*price_table.trade

In [21]:
price_table.p_L.sum()

-226.7669991017313

In [140]:
price_table.to_csv("price_table.csv")

In [23]:
stocks, quantity =robinhood.get_my_positions()

KeyError: 'results'